Tracking some rats for now... (data from new session, not already in gcloud)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from pathlib import Path

# Import Environment Variables
from dotenv import load_dotenv

# Import Utility Functions
from collab_env.data.file_utils import expand_path, get_project_root
from collab_env.data.gcs_utils import GCSClient

# Import Custom Scripts
from collab_env.tracking.alignment_gui import align_videos
from collab_env.tracking.model.local_model_inference import infer_with_yolo
from collab_env.tracking.model.local_model_tracking import (
    output_tracked_bboxes_csv,
    overlay_tracks_on_video,
    plot_tracks_at_frame_bbox_from_video,
    run_tracking,
    visualize_detections_from_video,
)
from collab_env.tracking.thermal_processing import (
    process_directory,
    validate_session_structure,
)
import shutil
import cv2
import numpy as np


In [ ]:
# split raw folder into session folders, and convert to mp4, and compute max projection image
BASE_DIR = '/Users/emily/Downloads/'
DATE_STR = '2025_07_17'
RAW_DIR = os.path.join(BASE_DIR, DATE_STR)
csq_dir = os.path.join(RAW_DIR, "Thermal_1")
csq_files = list(Path(csq_dir).glob("*.csq"))
for idx, csq in enumerate(csq_files, start=1):
    # Create session directory
    session_dir = os.path.join(BASE_DIR, f"{DATE_STR}-session_{idx:04d}")
    # check if session directory already exists
    if os.path.exists(session_dir):
        print(f"Session directory {session_dir} already exists. Skipping...")
        continue
    os.makedirs(session_dir, exist_ok=True)
    os.makedirs(os.path.join(session_dir, "thermal_1"), exist_ok=True)
    print(f"Moving {csq} to {os.path.join(session_dir, 'thermal_1', csq.name)}")
    shutil.copy(csq, os.path.join(session_dir, "thermal_1", csq.name))

    # Process each session directory
    process_directory(
        folder_path=os.path.join(session_dir),
        out_path=os.path.join(session_dir, "processed"),
        color="magma",  # 'Grays_r'
        preview=False,
        # max_frames=10,
        fps=30,
    )


        

Session directory /Users/emily/Downloads/2025_07_17-session_0001 already exists. Skipping...
Session directory /Users/emily/Downloads/2025_07_17-session_0002 already exists. Skipping...
Session directory /Users/emily/Downloads/2025_07_17-session_0003 already exists. Skipping...
Moving /Users/emily/Downloads/2025_07_17/Thermal_1/20250717202910847.csq to /Users/emily/Downloads/2025_07_17-session_0004/thermal_1/20250717202910847.csq
Found 1 .csq files in /Users/emily/Downloads/2025_07_17-session_0004/thermal_1

Processing [0] in thermal_1: 20250717202910847.csq
Auto-detecting vmin/vmax...
→ Using vmin=26.0, vmax=35.0
🖼️ Total frames in video: 2482


🔄 Writing frames: 100%|██████████| 2482/2482 [10:18<00:00,  4.01frame/s] 



✅ Exported 2482 frames to: /Users/emily/Downloads/2025_07_17-session_0004/processed/thermal_1/thermal_26_35.mp4
⏱ Approx. duration: 82.73 seconds at 30 fps
⚠️ Folder thermal_2 does not exist in /Users/emily/Downloads/2025_07_17-session_0004. Skipping...
Moving /Users/emily/Downloads/2025_07_17/Thermal_1/20250717210946836.csq to /Users/emily/Downloads/2025_07_17-session_0005/thermal_1/20250717210946836.csq
Found 1 .csq files in /Users/emily/Downloads/2025_07_17-session_0005/thermal_1

Processing [0] in thermal_1: 20250717210946836.csq
Auto-detecting vmin/vmax...
→ Using vmin=26.0, vmax=36.0
🖼️ Total frames in video: 42989


🔄 Writing frames:   0%|          | 92/42989 [00:26<5:50:32,  2.04frame/s]

In [4]:
def max_projection_from_video(video_path, out_path):
    """
    Compute the maximum projection image from a video file.
    
    Args:
        video_path (str): Path to the input video file.
        out_path (str): Path to save the maximum projection image.
    """
    cap = cv2.VideoCapture(video_path)
    ok, frame = cap.read()
    acc = frame.astype(np.uint16)
    while ok:
        acc = np.maximum(acc, frame)
        ok, frame = cap.read()
    cap.release()
    cv2.imwrite(out_path, np.clip(acc, 0, 255).astype(np.uint8))

In [7]:
# compute max projection image for each session

sessions_list = list(Path(BASE_DIR).glob(f"{DATE_STR}-session_*"))
for session_dir in sessions_list:
    print(f"Processing session directory: {session_dir}")
    max_proj_path = os.path.join(session_dir, "processed", "max_projection.png")
    # find the video file in the processed directory
    mp4_files = list(Path(os.path.join(session_dir, "processed", "thermal_1")).glob("*.mp4"))
    # check if there are any mp4 files
    if not mp4_files:
        print(f"No mp4 files found in {os.path.join(session_dir, 'processed', 'thermal_1')}")
        continue
    video_path = mp4_files[0] 
    max_projection_from_video(video_path, max_proj_path)
    # get just the folder name from the session directory
    session_str = os.path.basename(session_dir)
    max_projection_from_video(video_path, os.path.join(BASE_DIR, f'{session_str}-max_projection.png'))
    print(f"Max projection image saved to {max_proj_path}")


Processing session directory: /Users/emily/Downloads/2025_07_17-session_0007-max_projection.png
No mp4 files found in /Users/emily/Downloads/2025_07_17-session_0007-max_projection.png/processed/thermal_1
Processing session directory: /Users/emily/Downloads/2025_07_17-session_0004-max_projection.png
No mp4 files found in /Users/emily/Downloads/2025_07_17-session_0004-max_projection.png/processed/thermal_1
Processing session directory: /Users/emily/Downloads/2025_07_17-session_0002-max_projection.png
No mp4 files found in /Users/emily/Downloads/2025_07_17-session_0002-max_projection.png/processed/thermal_1
Processing session directory: /Users/emily/Downloads/2025_07_17-session_0001
Max projection image saved to /Users/emily/Downloads/2025_07_17-session_0001/processed/max_projection.png
Processing session directory: /Users/emily/Downloads/2025_07_17-session_0001-max_projection.png
No mp4 files found in /Users/emily/Downloads/2025_07_17-session_0001-max_projection.png/processed/thermal_1
P